In [28]:
import pandas as pd
import mysql.connector as sql
import math

In [29]:
def term(x,y):
    try:
        bond_term = int(x.year) - int(y.year)
    except:
        bond_term = 100
    return bond_term

In [30]:
def Type_term(x):
    if x >= 10:
        return 'Long Term'
    elif x < 10 and x >= 5:
        return  'Mid Term'
    else:
        return 'Short Term'

In [31]:
def assign_rating(x):
    x = float(x)
    if math.isnan(x):
        return 'UN'
    else:
        if x < 0:
            return 'NR'
        elif x<=13.5:
            return "Junk"
        elif x>13.5 and x<=16.5:
            return 'Lower medium grade'
        elif x>16.5 and x<=19.5:
            return 'Upper medium grade'
        elif x>19.5 and x<=22.5:
            return 'High grade'
        else:
            return 'Prime'

In [32]:
def get_all_bonds_in_list(start_date,end_date):
    print('start to get data from {} to {}'.format(start_date,end_date))
    #####Create engine:
    db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
    ##security query
    security_query = "select * from security_info"
    ##read data
    security_data = pd.read_sql(security_query,con=db_connection)
    security_data = security_data[['SecurityID','Currency','IssueDate','MaturityDate']]
    ##price query
    price_query = "select * from bond_spread where  KeyDate between '{}' and '{}'".format(start_date,end_date)
    #####read bond_data from db:
    final_data = pd.read_sql(price_query,con=db_connection)
    final_data = final_data.merge(security_data, on=['SecurityID'], how='left')
    final_data.dropna(subset=["ZSpread"],inplace=True)
    final_data_1 = final_data[final_data.Currency == 'USD']
    final_data = final_data_1.groupby('SecurityID')
    bonds_list = [final_data.get_group(x) for x in final_data.groups]
    db_connection.close()
    return bonds_list

In [33]:
def merge_left_right(data1,data2):
    temp_data = data1.append(data2)
    return temp_data

def merg_sort(list_data):
    list_length = len(list_data)
    if list_length == 1:
        output = list_data[0][2]
        return output
    else:
        mid_point = math.floor(list_length/2)
        #print(len(list_data[:mid_point]))
        left = list_data[mid_point:]
        right = list_data[:mid_point]
        return merge_left_right(merg_sort(left), merg_sort(right))

In [34]:
silding_windows = [['2018-12-31','2019-02-28'],['2019-03-01','2019-04-30'],
                   ['2019-05-01','2019-06-30'],['2019-07-01','2019-08-31'],
                   ['2019-09-01','2019-10-31'],['2019-11-01','2019-12-31']]

## Cluster data

In [35]:
cluster_data = pd.read_csv('Cluster_group_125.csv')

In [36]:
cluster_data.drop(columns=['Unnamed: 0'],inplace=True)

In [37]:
cluster_data.head()

,SecurityID,2019-02-28,2019-04-30,2019-06-30,2019-08-31,2019-10-31,2019-12-31
0,3051.0,Cluster 0,Cluster 85,Cluster 70,Cluster 82,Cluster 99,Cluster 91
1,3704.0,Cluster 0,Cluster 119,Cluster 0,Cluster 12,Cluster 65,Cluster 87
2,4121.0,Cluster 0,Cluster 52,Cluster 53,Cluster 72,Cluster 6,Cluster 91
3,4130.0,Cluster 0,Cluster 76,Cluster 124,Cluster 89,Cluster 10,Cluster 109
4,4259.0,Cluster 0,Cluster 95,Cluster 28,Cluster 86,Cluster 105,Cluster 76


In [38]:
def get_the_daily_spread_windows(bonds_list):
    bond_spread_list = []
    for bond in bonds_list:
        try:
            bond['G_change']=bond.GSpread.pct_change()
            bond['ModifiedDuration_Plain_change']=bond.ModifiedDuration_Plain.pct_change()
            bond['YieldWorst_change']=bond.YieldWorst.pct_change()
            bond.dropna(subset=['G_change'],inplace = True)
            bond_spread_list.append((len(bond.GSpread.values),bond.SecurityID.iloc[0],bond))
        except:
            pass
        return bond_spread_list

In [39]:
def Average_daily_Gspread_change_cluster(window,cluster_data):
    ####Grab Data###
    bond_list1 = get_all_bonds_in_list(window[0],window[1])
    ####G Change ###
    daily_spread_1 = get_the_daily_spread_windows(bond_list1)
    new_data = merg_sort(daily_spread_1)
    new_data.dropna(inplace=True)
    data1 = new_data.merge(cluster_data[['SecurityID',window[1]]],on=['SecurityID'],how ='left')
    temp = data1.groupby(by=['KeyDate',window[1]],as_index=False).mean()
    data1['Group'] = data1[window[1]] 
    data1.drop(columns=[window[1]],inplace=True)
    temp['Group'] = temp[window[1]]
    end = temp[['Group','KeyDate','G_change','YieldWorst_change','ModifiedDuration_Plain_change']]
    end = end.rename(columns={'G_change':'Cluster_G_change','YieldWorst_change':'Cluster_average_YieldWorst_change','ModifiedDuration_Plain_change':'Cluster_average_ModifiedDuration_Plain_change'})
    return end,data1

## Read Your secuirty data here

In [40]:
db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
security_query = "select * from security_info"
security_data = pd.read_sql(security_query,con=db_connection)

## END

## Read Your Rating data

In [41]:
db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
rating_query = "select * from rating"
rating_data = pd.read_sql(rating_query,con=db_connection)

In [42]:
rating_data = rating_data.groupby(by=['SecurityID'],as_index=False).mean()

In [43]:
security_data['Term'] = security_data.apply(lambda x:term(x.MaturityDate,x.IssueDate),axis=1)

In [44]:
security_data['Term'] = security_data['Term'].fillna(100)
security_data['Type of Term'] = security_data['Term'].apply(Type_term)

In [45]:
security_data = security_data.merge(rating_data,on =['SecurityID'],how='left')

In [46]:
security_data['Rating'] = security_data['RatingSP'].apply(assign_rating)

In [47]:
security_data.columns

Index(['SecurityID', 'OfficialName', 'SecurityType', 'SecuritySubType',
       'SecurityBBGType', 'SecurityBBGYellowKey', 'Series', 'SecuritySector',
       'SecurityIndustry', 'SecuritySubIndustry', 'BBGSecuritySector',
       'IssuerParent', 'Obligor', 'Ticker', 'EquityTicker', 'Issuer',
       'ObligorID', 'CountryDomicile', 'CountryRisk', 'CountryIssue',
       'Currency', 'Coupon', 'CouponType', 'CouponFrequency', 'CouponDayCount',
       'CreationDate', 'LastModifiedTime', 'IssueDate', 'PricingDate',
       'MaturityDate', 'IsCallable', 'IsConvertible', 'IsContingent',
       'IsFactorable', 'IsHybrid', 'IsMaple', 'IsPrivatePlacement',
       'IsPreliminary', 'IsPerpetual', 'IsJunior', 'Seniority', 'PaymentRank',
       'IsBailIn', 'QuoteSize', 'AmountIssued', 'MinIncrement', 'MinPiece',
       'BBGDescription', 'Term', 'Type of Term', 'RatingSP', 'Rating'],
      dtype='object')

In [48]:
def get_the_cluster_data(security_info,cluster_data,windows):
    print('start to get data for {}'.format(windows[1]))
    security_info = security_data[['SecurityID','SecuritySector','AmountIssued','Coupon','Seniority','Term','RatingSP','Rating','Type of Term']]
    a = security_info.merge(cluster_data[['SecurityID',windows[1]]],on=['SecurityID'],how='left')
    a.dropna(subset=[windows[1]],inplace=True)
    time_period = a.rename(columns={'2019-02-28':'Group'})

    Group_Seniority = time_period.groupby(by=['Group','Seniority']).count()
    Group_sector = time_period.groupby(by=['Group','SecuritySector']).count()
    Group_Type_term = time_period.groupby(by=['Group','Type of Term']).count()
    Group_Rating = time_period.groupby(by=['Group','Rating']).count()
    
    Group = time_period.groupby(by=['Group']).count()
    Sector = time_period.groupby(by=['SecuritySector']).count()
    Type_of_Term = time_period.groupby(by=['Type of Term']).count()
    Seniority = time_period.groupby(by=['Seniority']).count()
    Rating = time_period.groupby(by=['Rating']).count()
    
    Sector_per = Sector/len(time_period)
    Type_of_Term_per = Type_of_Term/len(time_period)
    Seniority_per = Seniority/len(time_period)
    Rating_per = Rating/len(time_period)

    Group_sector_per = Group_sector/Group
    Group_Seniority_per = Group_Seniority/Group
    Group_type_term_per = Group_Type_term/Group
    Group_rating_per =  Group_Rating/Group

    Group_Sector_indicator = (Group_sector_per - Sector_per)*100
    Group_Seniority_indicator = (Group_Seniority_per - Seniority_per)*100
    Group_type_term_indicator = (Group_type_term_per - Type_of_Term_per)*100
    Group_rating_indicator= (Group_rating_per - Rating_per)*100

    for i in range(0,125):
        if i == 0:
            f = Group_Seniority_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            f['Group'] ='Cluster {}'.format(i)
        else:
            temp = Group_Seniority_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            temp['Group'] ='Cluster {}'.format(i)
            f = f.merge(temp,how='outer').fillna(0)
    for i in range(0,125):
        if i == 0:
            f1 = Group_Sector_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            f1['Group'] ='Cluster {}'.format(i)
        else:
            temp1 = Group_Sector_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            temp1['Group'] ='Cluster {}'.format(i)
            f1 = f1.merge(temp1,how='outer').fillna(0)
    for i in range(0,125):
        if i == 0:
            f2 = Group_type_term_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            f2['Group'] ='Cluster {}'.format(i)
        else:
            temp2 = Group_type_term_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            temp2['Group'] ='Cluster {}'.format(i)
            f2 = f2.merge(temp2,how='outer').fillna(0)
    for i in range(0,125):
        if i == 0:
            f3 = Group_rating_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            f3['Group'] ='Cluster {}'.format(i)
        else:
            temp3 = Group_rating_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            temp3['Group'] ='Cluster {}'.format(i)
            f3 = f3.merge(temp3,how='outer').fillna(0)
    f= f.merge(f1,on='Group',how = 'left')
    f= f.merge(f2,on='Group',how = 'left')
    f= f.merge(f3,on='Group',how = 'left')
    b = time_period.groupby(by=['Group'],as_index=False).mean()
    b.drop(columns=['SecurityID'],inplace=True)
    b = b.rename(columns={'AmountIssued':'Cluster_average_AmountIssued','Coupon':'Cluster_average_Coupon','Term':'Cluster_average_Term','RatingSP':'Cluster_average_Rating'})
    window_cluster_data = b.merge(f,on='Group',how = 'left')
    return window_cluster_data

In [49]:
from sqlalchemy import create_engine

In [50]:
cluster_info = get_the_cluster_data(security_data,cluster_data,silding_windows[0])

start to get data for 2019-02-28


In [51]:
cluster_info

,Group,Cluster_average_AmountIssued,Cluster_average_Coupon,Cluster_average_Term,Cluster_average_Rating,Jr Subordinated Unsecured,Secured,Sr Unsecured,Subordinated Unsecured,1st Lien Secured,...,Technology,Long Term,Mid Term,Short Term,High grade,Junk,Lower medium grade,UN,Upper medium grade,Prime
0,Cluster 0,6.060661e+08,4.724531,17.824742,16.180732,-0.014627,0.024537,0.031187,0.001597,0.000000,...,0.003254,0.042336,-0.058375,0.016039,0.034805,-0.079984,-0.037977,-0.073532,0.163791,0.000000
1,Cluster 1,9.649194e+08,4.410567,15.305732,16.267418,-0.018567,-0.049138,0.096654,-0.024472,0.003707,...,0.044819,0.056191,-0.012804,-0.043387,0.024824,-0.086616,-0.016045,-0.032886,0.105087,0.005636
2,Cluster 10,7.517519e+08,4.728494,13.470588,14.749780,-0.013172,-0.032953,0.044649,0.008874,0.000785,...,0.044369,0.020747,-0.011680,-0.009067,-0.034449,0.115407,-0.111961,-0.060312,0.098418,0.000000
3,Cluster 100,6.369403e+08,4.613108,15.387097,16.222335,0.000000,0.071538,-0.018475,-0.006939,-0.013004,...,-0.016478,-0.014420,-0.015475,0.029896,0.006537,-0.127983,-0.013922,-0.019325,0.151043,0.003650
4,Cluster 101,6.455059e+08,4.440804,14.774510,16.156443,-0.015132,-0.009423,0.009355,0.018678,-0.005098,...,-0.018376,0.097218,-0.060700,-0.036518,-0.008959,-0.145377,0.074313,-0.005410,0.092536,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Cluster 95,5.961280e+08,5.362838,16.087500,14.619028,0.037564,0.019253,-0.089174,0.037550,0.002991,...,-0.012983,0.061924,-0.018298,-0.043626,-0.020479,0.048496,0.015245,0.041158,-0.077317,0.000000
121,Cluster 96,7.269452e+08,4.357861,12.934307,16.244963,-0.017637,0.004746,0.054439,-0.006154,-0.027210,...,-0.023385,-0.017638,0.008071,0.009567,0.007715,-0.111815,-0.004190,-0.061944,0.177336,0.000000
122,Cluster 97,7.667758e+08,5.426098,17.151515,13.455243,0.028094,-0.007641,-0.033871,0.018232,-0.011782,...,-0.007680,-0.039591,0.072990,-0.033399,-0.020100,0.164783,-0.066952,0.052522,-0.123150,0.000000
123,Cluster 98,6.052778e+08,5.013682,16.270588,15.925284,-0.001407,0.049400,-0.025939,0.008874,0.000000,...,-0.026219,0.020747,-0.046974,0.026227,-0.046214,-0.084593,-0.041373,0.010276,0.157242,0.004662


In [52]:
security_info = security_data[['SecurityID','SecuritySector','AmountIssued','Coupon','Seniority','Term','RatingSP','Rating','Type of Term']]
a = security_info.merge(cluster_data[['SecurityID','2019-02-28']],on=['SecurityID'],how='left')
a.dropna(subset=['2019-02-28'],inplace=True)
time_period = a.rename(columns={'2019-02-28':'Group'})
Group_Seniority = time_period.groupby(by=['Group','Seniority']).count()
Group_sector = time_period.groupby(by=['Group','SecuritySector']).count()
Type_term = time_period.groupby(by=['Group','Type of Term']).count()
Group = time_period.groupby(by=['Group']).count()
Sector = a.groupby(by=['SecuritySector']).count()

In [53]:
Group_sector_per = Group_sector/len(time_period)

In [54]:
cluster_info.to_csv('test1.csv')